In [6]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
import csv
import itertools
import os
import random
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, KFold
from sklearn.kernel_ridge import KernelRidge
from sympy import symbols, simplify, lambdify, Function, diff, Mul
from sklearn.metrics import mean_squared_error
from cvxopt import matrix, solvers
from qpsolvers import solve_qp
from scipy.sparse import csc_matrix
import math
from scipy.stats import pearsonr
from itertools import combinations_with_replacement
from sympy import symbols, Matrix
from scipy import sparse
import ast
import cvxpy as cp
d = int(input("Enter polynomial kernel d = "))
c = int(input("Enter c =  "))
def compute_kernel_matrix(X, c, d):
    n = X.shape[0]
    K = (c + np.dot(X, X.T)) ** d
    return K
def solve_for_lambda(data, c, d, lambda_value):
    B501_data = pd.read_csv('../../../Data/FPUT\FPUT2/B501.csv')
    traj_len = B501_data.groupby('trajectory').size()
    rep = int(round(traj_len.mean()))
    K = compute_kernel_matrix(data, c, d)
    I = np.eye(K.shape[0])
    K_with_I = K + lambda_value * I
    K_with_I_inv = np.linalg.inv(K_with_I)
    m = 5 # number of trajectory
    num_y_variables = m
    y_symbols = [sp.symbols(f'y{i}') for i in range(num_y_variables)]
    y_pattern = [sp.symbols(f'y{i}') for i in range(m)]
    y_repeated = np.repeat(y_pattern, rep, axis=0)
    y = sp.Matrix([sp.symbols(f'y{i}') for i in range(m)])  
    M_matrix = sp.Matrix(K_with_I_inv)
    n = K.shape[0]
    M = sp.zeros(n, m)
    for i in range(n):
        for j in range(m):
            start_idx = j * rep
            end_idx = (j + 1) * rep
            M[i, j] = sp.Add(*M_matrix[i, start_idx:end_idx])
    M_transpose = M.transpose()
    A = M_transpose @ M 
    A_np = np.array(A).astype(np.float64)
    return A_np
m = 5 # number of trajectory
base_path = "../../../Data/FPUT\FPUT2/"
file_names = [f"B50{i}.csv" for i in range(1, m+1)]
filenames = [base_path + file_name for file_name in file_names]
data_list = [np.loadtxt(filename, delimiter=',', skiprows=1, usecols=(0, 1, 2, 3, 4)) for filename in filenames]
data, data2, data3, data4, data5 = data_list
solutions_combined = []
for i in range(0, 8):
    lambda_value = 10**(-i)
    A_np_list = [solve_for_lambda(data_array, c=c, d=d, lambda_value=lambda_value) for data_array in data_list]
    A1_np, A2_np, A3_np, A4_np, A5_np = A_np_list
    As_p = np.mean(A_np_list, axis=0)
    P = np.array(As_p)
    q = np.zeros(m)
    a = np.random.uniform(-3, 3, m)
    G = np.zeros((0, m))
    h = np.zeros(0)
    A = a.reshape(1, -1)
    P = csc_matrix(P)
    G = csc_matrix(G)
    A = csc_matrix(A) 
    b = np.array([1.0])
    y = solve_qp(P, q, G, h, A, b, solver="clarabel")
    y_opt = y.flatten() if isinstance(y, np.ndarray) else np.array(y).flatten()
    y_names = [sp.symbols(f'y{i}') for i in range(m)]
    y_dict = {name: value for name, value in zip(y_names, y_opt)}
    solutions_combined.append((lambda_value, y_dict))
def compute_kernel_matrix(X, c, d):
    n = X.shape[0]
    K = (c + np.dot(X, X.T)) ** d
    return K
def solve_for_lambda(data, c, d, lambda_value, y_values):
    K = compute_kernel_matrix(data, c, d)
    I = np.eye(K.shape[0])
    K_with_I = K + lambda_value * I
    K_with_I_inv = np.linalg.inv(K_with_I)
    y_repeated = np.repeat(y_values, len(data) // len(y_values))
    alpha_sym = K_with_I_inv @ y_repeated
    return K_with_I_inv, alpha_sym
def generate_f_alpha_expression(alpha_sym, x1, x2, x3, x4, x5, x_q1_sym, x_q2_sym, x_q3_sym, x_q4_sym, x_q5_sym, c, d):
    f_alpha = 0
    for i in range(len(alpha_sym)):
        f_alpha += alpha_sym[i] * (c + (x1[i] * x_q1_sym) + (x2[i] * x_q2_sym) + (x3[i] * x_q3_sym) + (x4[i] * x_q4_sym) + (x5[i] * x_q5_sym)) ** d
    f_alpha_expanded = sp.expand(f_alpha)
    f_alpha_collected = sp.collect(f_alpha_expanded, (x_q1_sym, x_q2_sym, x_q3_sym, x_q4_sym, x_q5_sym))
    return f_alpha_collected
def process_dataset(file_path, c, d, lambda_values, y_values_dicts):
    data = np.loadtxt(file_path, delimiter=',', skiprows=1, usecols=(0, 1, 2, 3, 4))
    x1, x2, x3, x4, x5 = data[:, 0], data[:, 1], data[:, 2], data[:, 3], data[:, 4]
    K_with_I_inv_list = []
    alpha_sym_list = []
    f_alpha_expression_list = []
    for lambda_val, (_, y_values_dict) in zip(lambda_values, y_values_dicts):
        K_with_I_inv, alpha_sym = solve_for_lambda(data, c, d, lambda_val, list(y_values_dict.values()))
        x_q1_sym, x_q2_sym, x_q3_sym, x_q4_sym, x_q5_sym = sp.symbols('x1 x2 x3 x4 x5')
        K_with_I_inv_list.append(K_with_I_inv)
        alpha_sym_list.append(alpha_sym)
        f_alpha_expression = generate_f_alpha_expression(alpha_sym, x1, x2, x3, x4, x5, x_q1_sym, x_q2_sym, x_q3_sym, x_q4_sym, x_q5_sym, c, d)
        f_alpha_expression_list.append(f_alpha_expression)
    
    return K_with_I_inv_list, alpha_sym_list, f_alpha_expression_list
lamda = 8
lambda_values = [10**(-i) for i in range(lamda)]
y_values_dicts_list = solutions_combined
base_path = "../../../Data/FPUT\FPUT2/"
file_names = [f"B50{i}.csv" for i in range(1, 6)]
file_paths = [base_path + file_name for file_name in file_names]
datasets = [pd.read_csv(path) for path in file_paths]
B501_data = datasets[0]
dr1 = datasets[-1]
traj_len = B501_data.groupby('trajectory').size()
rep4 = int(round(traj_len.mean()))
results = [process_dataset(path, c, d, lambda_values, y_values_dicts_list) for path in file_paths]
f_vectors = []
for _, _, f_alpha_expr in results:
    f_vector = [[] for _ in range(len(f_alpha_expr))]
    for index, row in dr1.iterrows():
        for i, expr in enumerate(f_alpha_expr):
            value = eval(str(expr), globals(), row.to_dict())
            f_vector[i].append(value)
    f_vectors.append(f_vector)
h_values = [list(y_dict[1].values()) for y_dict in y_values_dicts_list]
y_B_values = []
for h in h_values:
    y_B_values.extend([np.repeat(h, rep4) for _ in range(8)])
rmse_values = {}
for i, f_vector in enumerate(f_vectors):
    rmse_values[f"B50{i+1}"] = [np.sqrt(mean_squared_error(y, f)) for y, f in zip(y_B_values[i*8:(i+1)*8], f_vector)]
min_rmse_info = {}
for key, rmse_list in rmse_values.items():
    min_rmse = min(rmse_list)
    min_index = rmse_list.index(min_rmse)
    min_rmse_info[key] = (min_rmse, min_index)
min_rmse = float('inf')  
min_index = None
dataset_key = None
for key, rmse_list in rmse_values.items():
    for index, value in enumerate(rmse_list):
        if value < min_rmse:
            min_rmse = value
            min_index = index
            dataset_key = key
file_index = min_index // 8 + 1
sub_index = min_index % 8
lambda_values = [10**(-i) for i in range(lamda)]
h_value = h_values[-1]
# h_value = h_values[sub_index]
df2 = pd.read_csv('../../../Data/FPUT\FPUT2/trainingp_data50.csv')
m = df2['trajectory'].nunique()
df2['trajectory'] = df2['trajectory'].replace({i: h_value[i-1] for i in range(1, m+1)})
X_train = df2.iloc[:, :-1]
y_train = df2.iloc[:, -1]
X_train.to_csv('../../../results/FPUT\FPUT2/X_train.csv', index=False)
y_train = y_train.astype(float)
X_train = X_train.astype(float)
def polynomial_kernel(X, Y, degree=d):
    return (1 + np.dot(X, Y.T)) ** degree
param_grid = {'alpha': [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1]}
cv = KFold(n_splits=5, shuffle=True, random_state=42)
kr_model = KernelRidge(kernel=polynomial_kernel)
grid_search = GridSearchCV(kr_model, param_grid, cv=cv, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
class KernelMethodBase(object):
    '''
    Base class for kernel methods models
    Methods
    ----
    fit
    predict
    fit_K
    predict_K
    '''
    kernels_ = {
        'polynomial': polynomial_kernel,
    }
    def __init__(self, kernel='polynomial', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        self.fit_intercept_ = False
    def get_kernel_parameters(self, **kwargs):
        params = {}
        params['degree'] = kwargs.get('degree', d)
        return params
    def fit_K(self, K, y, **kwargs):
        pass
    def decision_function_K(self, K):
        pass
    def fit(self, X, y, **kwargs):
        self.X_train = X
        self.y_train = y
        K = self.kernel_function_(self.X_train, self.X_train, **self.kernel_parameters)
        return self.fit_K(K, y, **kwargs)
    def decision_function(self, X):
        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)
        return self.decision_function_K(K_x)
    def predict(self, X):
        pass
    def predict_K(self, K):
        pass
class KernelRidgeRegression(KernelMethodBase):
    '''
    Kernel Ridge Regression
    '''
    def __init__(self, alpha=0.1, **kwargs):
        self.alpha = alpha
        super(KernelRidgeRegression, self).__init__(**kwargs)
    def fit_K(self, K, y):
        n = K.shape[0]
        assert (n == len(y))
        A = K + self.alpha*np.identity(n)
        self.eta = np.linalg.solve(A , y)
        return self
    def decision_function_K(self, K_x):
        return K_x.dot(self.eta)
    def predict(self, X):
        return self.decision_function(X)
    def predict_K(self, K_x):
        return self.decision_function_K(K_x)
kernel = 'polynomial'
kr_model = KernelRidgeRegression(
    kernel=kernel,
    alpha=grid_search.best_params_['alpha'],
    )
kr_model.fit(X_train, y_train)
eta = kr_model.eta
x1, x2, x3, x4, x5 = sp.symbols('x1 x2 x3 x4 x5') 
polynomial_kernel = (1 + x1*sp.Symbol('xi1') + x2*sp.Symbol('xi2') + x3*sp.Symbol('xi3') + x4*sp.Symbol('xi4') + x5*sp.Symbol('xi5'))**d
f_beta = 0
for i in range(len(X_train)):
    f_beta += eta[i] * polynomial_kernel.subs({'xi1': X_train.iloc[i][0], 'xi2': X_train.iloc[i][1], 'xi3': X_train.iloc[i][2], 'xi4': X_train.iloc[i][3], 'xi5': X_train.iloc[i][4]})
candidate_CL = sp.expand(f_beta)
print("Candidate Conservation Law:")
sp.pprint(candidate_CL)
print("")
print("")
coefficients = list(candidate_CL.as_coefficients_dict().values())
terms = list(candidate_CL.as_coefficients_dict().keys())
filtered_terms = [term for coeff, term in zip(coefficients, terms) if abs(coeff) > 0.01]
filtered_ex = sum(Mul(coeff, term) for coeff, term in zip(coefficients, terms) if term in filtered_terms)
print("Final Conservation Law:")
sp.pprint(filtered_ex)
print("")
print("")
with open("../../../results/FPUT\FPUT2/candidate_CL.txt", "w") as file:
    file.write(str(candidate_CL))
with open("../../../results/FPUT\FPUT2/candidate_CL.txt", "r") as file:
    candidate_CL = sp.sympify(file.read())
df3 = pd.read_csv('../../../Data/FPUT\FPUT2/holdoutp_data50.csv')
traj_len = df3.groupby('trajectory').size()
rep1 = int(round(traj_len.mean()))
expression = sp.lambdify((x1, x2, x3, x4, x5), candidate_CL, "numpy")
df3['lamhold'] = expression(df3['x1'], df3['x2'], df3['x3'], df3['x4'], df3['x5'])
da = {'y{}'.format(i): h_value[i] for i in range(len(h_value))}
df3['Coluh(lamhold)'] = [da[f'y{i}'] for i in range(m) for _ in range(rep1)]
columns_to_compare = [('lamhold', 'Coluh(lamhold)')]
for col1, col2 in columns_to_compare:
    rmse = np.sqrt(mean_squared_error(df3[col1], df3[col2]))
    print(f'Generalisation Error (RMSE): {rmse}')
    print("")
with open("../../../results/FPUT\FPUT2/candidate_CL.txt", "r") as file:
    candidate_CL = sp.sympify(file.read())
f = sp.lambdify((x1, x2, x3, x4, x5), candidate_CL, "numpy")
dat = pd.read_csv('../../../Data/FPUT\FPUT2/holdoutp_data50.csv')
trajectories = dat['trajectory'].unique()
total_sum_squared_normalized_functional_value = 0
total_data_points = 0
num_x_variables = 5
for trajectory in trajectories:
    trajectory_data = dat[dat['trajectory'] == trajectory].copy()  
    cols = ['x' + str(i) for i in range(1, num_x_variables + 1)] # number of variable
    trajectory_data['functional_value'] = f(*trajectory_data[cols].values.T)
    mean_value = trajectory_data['functional_value'].mean()
    trajectory_data['functional_value_minus_mean'] = trajectory_data['functional_value'] - mean_value
    trajectory_data['normalized_functional_value'] = trajectory_data['functional_value_minus_mean'] / mean_value
    trajectory_data['squared_normalized_functional_value'] = trajectory_data['normalized_functional_value'] ** 2
    total_sum_squared_normalized_functional_value += trajectory_data['squared_normalized_functional_value'].sum()
    total_data_points += len(trajectory_data)
average_squared_normalized_functional_value = total_sum_squared_normalized_functional_value / total_data_points
standard_deviation = math.sqrt(average_squared_normalized_functional_value)
print(" Relative deviation:", standard_deviation)
print("")

Enter polynomial kernel d = 3
Enter c =  1
Candidate Conservation Law:
                        3                         2                          2
- 3.73096756746405e-7⋅x₁  + 6.66712631731264e-7⋅x₁ ⋅x₂ - 2.561245965918e-5⋅x₁ 

                            2                          2                      
⋅x₃ - 2.56971250794805e-5⋅x₁ ⋅x₄ + 0.136124858570681⋅x₁ ⋅x₅ - 2.36793637371779

      2                            2                                          
e-6⋅x₁  - 2.13778239364221e-7⋅x₁⋅x₂  + 5.1258927026572e-5⋅x₁⋅x₂⋅x₃ + 5.1430223

                                                                              
5880195e-5⋅x₁⋅x₂⋅x₄ - 0.272249691156422⋅x₁⋅x₂⋅x₅ + 4.70538061288911e-6⋅x₁⋅x₂ -

                          2                                                   
 1.97619639680235e-8⋅x₁⋅x₃  + 2.85729167947418e-7⋅x₁⋅x₃⋅x₄ + 4.5187538948166e-

                                                                 2            
7⋅x₁⋅x₃⋅x₅ - 7.0253017915578e-7⋅x₁⋅x₃ - 5.1376211168707

In [7]:
##### Second Search

def solve_for_lambda(data, c, d, lambda_value):
    B5d = pd.read_csv('../../../Data/FPUT\FPUT2/trainingp_data50.csv')
    traj_len = B5d.groupby('trajectory').size()
    rep = int(round(traj_len.mean()))
    K = compute_kernel_matrix(data, c, d)
    I = np.eye(K.shape[0])
    K_with_I = K + lambda_value * I
    K_with_I_inv = np.linalg.inv(K_with_I)
    m = 5 # number of trajectory
    num_y_variables = m
    y_symbols = [sp.symbols(f'y{i}') for i in range(num_y_variables)]
    y_pattern = [sp.symbols(f'y{i}') for i in range(m)]
    y_repeated = np.repeat(y_pattern, rep, axis=0)
    y = sp.Matrix([sp.symbols(f'y{i}') for i in range(m)])  
    M_matrix = sp.Matrix(K_with_I_inv)
    n = K.shape[0]
    M = sp.zeros(n, m)
    for i in range(n):
        for j in range(m):
            start_idx = j * rep
            end_idx = (j + 1) * rep
            M[i, j] = sp.Add(*M_matrix[i, start_idx:end_idx])
    M_transpose = M.transpose()
    A = M 
    F =A.T
    return A, K_with_I_inv, F
data = np.loadtxt('../../../Data/FPUT\FPUT2/trainingp_data50.csv', delimiter=',', skiprows=1, usecols=(0, 1, 2, 3, 4))
solutions_combined = []
for i in range(7, 8):
    lambda_value = 10**(-i)
    A, K_with_I_inv, F = solve_for_lambda(data, c=1, d=3, lambda_value=lambda_value)
def multilinear_coefficient(n, *ks):
    numerator = math.factorial(n)
    denominator = 1
    for k in ks:
        denominator *= math.factorial(k)
    return numerator // denominator
def multilinear_expansion(variables, n, row):
    expansions = []
    for ks in itertools.product(range(n + 1), repeat=len(variables)):
        if sum(ks) == n:
            coefficient = multilinear_coefficient(n, *ks)
            values = [row[var] ** k if var != '1' else 1 for var, k in zip(variables, ks)]
            term_value = coefficient * math.prod(values)
            expansions.append(term_value)
    return expansions[::-1]
def multilinear_expansion1(variables1, n1):
    expansions1 = []  
    for ks in itertools.product(range(n1 + 1), repeat=len(variables1)):
        if sum(ks) == n1:
            terms1 = [f"{var}**{k}" if k != 0 else f"{var}" for var, k in zip(variables1, ks) if k != 0]
            term1 = " * ".join(terms1)
            expansions1.append(term1)
    return expansions1[::-1]
def generate_inner_products(coefficients, terms):
    inner_products = [f"{c}*{t}" for c, t in zip(coefficients, terms)]
    return inner_products
data = pd.read_csv('../../../Data/FPUT\FPUT2/trainingp_data50.csv')
n = 3
data['c'] = 1
num_x_variables = 5
variables = ['c'] + [f'x{i}' for i in range(1, num_x_variables + 1)]
n1 = 3
variables1 = ['1'] + [f'x{i}' for i in range(1, num_x_variables + 1)]
all_entries = []
for index, row in data.iterrows():
    result = multilinear_expansion(variables, n, row)
    result1 = multilinear_expansion1(variables1, n1)
    expressions = generate_inner_products(result, result1)
    all_entries.append(result)
matrix = np.array(all_entries)
def generate_symbolic_terms(degree=3):
    terms = []
    variables = [x1, x2, x3, x4, x5]
    terms.append('1**1')
    for d in range(1, degree + 1):
        for comb in combinations_with_replacement(variables, d):
            term = ' * '.join([f'{v}**{comb.count(v)}' for v in set(comb)])
            terms.append(term)
    return terms
def print_as_single_row(terms):
    matrix_row = "  ".join(terms)
    shape = (1, len(terms)) 
result = generate_symbolic_terms(degree=3)
print_as_single_row(result)
F_numpy = np.array(F.tolist()).astype(np.float64)
if matrix.shape[1] == F_numpy.shape[0]:
    result = np.dot(F_numpy, matrix)
ds=np.dot(F_numpy, matrix)
shape = print_as_single_row(result)
symbolic_matrix = sp.Matrix(1, len(result), result)
ds_symbolic = ds * symbolic_matrix.T 
shape = print_as_single_row(result)
symbolic_matrix = sp.Matrix(1, len(result), result)
ds_symbolic = ds * symbolic_matrix.T 
def compute_gradients(expr, variables):
    return [sp.diff(expr, var) for var in variables]
variables = [x1, x2, x3, x4, x5]
gradient_rows = []
for i in range(ds_symbolic.shape[0]):
    gradient_row = compute_gradients(ds_symbolic[i, 0], variables)
    gradient_rows.append(gradient_row)
gradients_matrix = sp.Matrix(gradient_rows)
N = 1
D = num_x_variables
lower_bound = -2
upper_bound = 2
points = np.random.uniform(lower_bound, upper_bound, (N, D))
rp = ', '.join(f"{x:.8f}" for x in points[0])
values = [float(x) for x in rp.split(', ')]
x1_value, x2_value, x3_value, x4_value, x5_value = values
P_J = gradients_matrix.subs({x1: x1_value, x2: x2_value, x3: x3_value, x4: x4_value, x5: x5_value})
P_J_matrix = Matrix(P_J)
rank_P_J = P_J_matrix.rank()
f = x3+x4
f_vector = sp.Matrix([f])
variables = sp.Matrix([x1, x2, x3, x4, x5])
grad_f1 = f_vector.jacobian(variables)
sp.pprint(grad_f1)
grad_f1_shape = grad_f1.shape
x1_value, x2_value, x3_value, x4_value, x5_value = values
P_J1 = grad_f1.subs({x1: x1_value, x2: x2_value, x3: x3_value, x4: x4_value, x5: x5_value})
P_J_matrix1 = Matrix(P_J1) 
rank_P_J1 = P_J_matrix1.rank() 
dJ = P_J_matrix1 * P_J_matrix 
def solve_for_lambda(data, c, d, lambda_value):
    B5d = pd.read_csv('../../../Data/FPUT\FPUT2/trainingp_data50.csv')
    traj_len = B5d.groupby('trajectory').size()
    rep = int(round(traj_len.mean()))
    K = compute_kernel_matrix(data, c, d)
    I = np.eye(K.shape[0])
    K_with_I = K + lambda_value * I
    K_with_I_inv = np.linalg.inv(K_with_I)
    m = 5 # number of trajectory
    num_y_variables = m
    y_symbols = [sp.symbols(f'y{i}') for i in range(num_y_variables)]
    y_pattern = [sp.symbols(f'y{i}') for i in range(m)]
    y_repeated = np.repeat(y_pattern, rep, axis=0)
    y = sp.Matrix([sp.symbols(f'y{i}') for i in range(m)])  
    M_matrix = sp.Matrix(K_with_I_inv)
    n = K.shape[0]
    M = sp.zeros(n, m)
    for i in range(n):
        for j in range(m):
            start_idx = j * rep
            end_idx = (j + 1) * rep
            M[i, j] = sp.Add(*M_matrix[i, start_idx:end_idx])
    M_transpose = M.transpose()
    A = M 
    F =A.T
    return A, K_with_I_inv, F
data = np.loadtxt('../../../Data/FPUT\FPUT2/trainingp_data50.csv', delimiter=',', skiprows=1, usecols=(0, 1, 2, 3, 4))
solutions_combined = []
for i in range(7, 8):
    lambda_value = 10**(-i)
    A, K_with_I_inv, F = solve_for_lambda(data, c=1, d=3, lambda_value=lambda_value)
a1 = np.random.uniform(-3, 3, num_x_variables)
def solve_for_lambda(data, c, d, lambda_value):
    B5d = pd.read_csv('../../../Data/FPUT\FPUT2/trainingp_data50.csv')
    traj_len = B5d.groupby('trajectory').size()
    rep = int(round(traj_len.mean()))
    K = compute_kernel_matrix(data, c, d)
    I = np.eye(K.shape[0])
    K_with_I = K + lambda_value * I
    K_with_I_inv = np.linalg.inv(K_with_I) 
    M_matrix = sp.Matrix(K_with_I_inv)
    n = K.shape[0]
    m = 5
    M = sp.zeros(n, m)
    for i in range(n):
        for j in range(m):
            start_idx = j * rep
            end_idx = (j + 1) * rep
            M[i, j] = sp.Add(*M_matrix[i, start_idx:end_idx])
    M_transpose = M.transpose()
    A = M_transpose @ M
    F =A
    return A, K_with_I_inv, F
data = np.loadtxt('../../../Data/FPUT\FPUT2/trainingp_data50.csv', delimiter=',', skiprows=1, usecols=(0, 1, 2, 3, 4))
solutions_combined = []
for i in range(7, 8):
    lambda_value = 10**(-i)
    A, K_with_I_inv, F = solve_for_lambda(data, c=1, d=3, lambda_value=lambda_value)
def solve_for_lambda(data, c, d, lambda_value):
    B5d = pd.read_csv('../../../Data/FPUT\FPUT2/trainingp_data50.csv')
    traj_len = B5d.groupby('trajectory').size()
    rep = int(round(traj_len.mean()))
    K = compute_kernel_matrix(data, c, d)
    I = np.eye(K.shape[0])
    K_with_I = K + lambda_value * I
    K_with_I_inv = np.linalg.inv(K_with_I)
    M_matrix = sp.Matrix(K_with_I_inv)
    n = K.shape[0]
    m = 5
    M = sp.zeros(n, m)
    for i in range(n):
        for j in range(m):
            start_idx = j * rep
            end_idx = (j + 1) * rep
            M[i, j] = sp.Add(*M_matrix[i, start_idx:end_idx])
    M_transpose = M.transpose()
    A = M_transpose @ M
    F = np.array(A.evalf()).astype(np.float64)
    return A, K_with_I_inv, F
N = 1
D = num_x_variables 
lower_bound = -2
upper_bound = 2
points = np.random.uniform(lower_bound, upper_bound, (N, D))
rp = ', '.join(f"{x:.8f}" for x in points[0])
values = [float(x) for x in rp.split(', ')]
x1_value, x2_value, x3_value, x4_value, x5_value = values
P_J = gradients_matrix.subs({x1: x1_value, x2: x2_value, x3: x3_value, x4: x4_value, x5: x5_value})
P_J_matrix = sp.Matrix(P_J)
rank_P_J = P_J_matrix.rank()
dJ = P_J_matrix1 * P_J_matrix
data = np.loadtxt('../../../Data/FPUT\FPUT2/trainingp_data50.csv', delimiter=',', skiprows=1, usecols=(0, 1, 2, 3, 4))
solutions_combined = []
for i in range(7, 8):
    lambda_value = 10**(-i)
    A, K_with_I_inv, F = solve_for_lambda(data, c=1, d=3, lambda_value=lambda_value)
    q = np.zeros(F.shape[0])
    G = sparse.csc_matrix((0, F.shape[0]))
    h = np.zeros(0)
    a = np.array([values], dtype=np.float64)
    dJ_numeric = np.array(dJ.evalf()).astype(np.float64)
    r = np.array([dJ_numeric], dtype=np.float64)
    F_sparse = sparse.csc_matrix(F)
    A_eq = sparse.csc_matrix(np.vstack([a.reshape(1, -1), r.reshape(1, -1)]))
    b_eq = np.array([1.0, 0.0], dtype=np.float64)
    y = solve_qp(F_sparse, q, G, h, A_eq, b_eq, solver="clarabel")
symbolic_matrix = sp.Matrix(1, len(result), result)
ds_symbolic = ds * symbolic_matrix.T 
y_values = np.array(y)
y = sp.Matrix(y_values)
pr = y.T * ds_symbolic
pr_expr = pr[0] 
SCL = str(pr_expr)
with open("../../../results/FPUT\FPUT2/SCL.txt", "w") as file:
    file.write(str(SCL))
with open("../../../results/FPUT\FPUT2/SCL.txt", "r") as file:
    SCL = sp.sympify(file.read())
f = sp.lambdify((x1, x2, x3, x4, x5), SCL, "numpy")
dat = pd.read_csv('../../../Data/FPUT\FPUT2/holdoutp_data50.csv')
trajectories = dat['trajectory'].unique()
total_sum_squared_normalized_functional_value = 0
total_data_points = 0
num_x_variables = 5
for trajectory in trajectories:
    trajectory_data = dat[dat['trajectory'] == trajectory].copy()  
    cols = ['x' + str(i) for i in range(1, num_x_variables + 1)] # number of variable
    trajectory_data['functional_value'] = f(*trajectory_data[cols].values.T)
    mean_value = trajectory_data['functional_value'].mean()
    trajectory_data['functional_value_minus_mean'] = trajectory_data['functional_value'] - mean_value
    trajectory_data['normalized_functional_value'] = trajectory_data['functional_value_minus_mean'] / mean_value
    trajectory_data['squared_normalized_functional_value'] = trajectory_data['normalized_functional_value'] ** 2
    total_sum_squared_normalized_functional_value += trajectory_data['squared_normalized_functional_value'].sum()
    total_data_points += len(trajectory_data)
average_squared_normalized_functional_value = total_sum_squared_normalized_functional_value / total_data_points
standard_deviation = math.sqrt(average_squared_normalized_functional_value)
print("Second CL:")
sp.pprint(SCL)
print("")
print("")
coefficients = list(SCL.as_coefficients_dict().values())
terms = list(SCL.as_coefficients_dict().keys())
filtered_terms = [term for coeff, term in zip(coefficients, terms) if abs(coeff) > 0.001]
filtered_ex = sum(Mul(coeff, term) for coeff, term in zip(coefficients, terms) if term in filtered_terms)
print("Final Conservation Law:")
sp.pprint(filtered_ex)
print("")
print("")
print(" Relative deviation:", standard_deviation)
print("")

[0  0  1  1  0]
Second CL:
                     3                         2                           2  
1.9812305830327e-7⋅x₁  - 1.02684955343039e-5⋅x₁ ⋅x₂ + 0.0030216529125727⋅x₁ ⋅x

                          2                          2                        
₃ + 0.00303707801716749⋅x₁ ⋅x₄ - 0.439457323297892⋅x₁ ⋅x₅ + 0.0005854443578691

     2                            2                                           
55⋅x₁  + 1.55567062230308e-5⋅x₁⋅x₂  - 0.00605189667404435⋅x₁⋅x₂⋅x₃ - 0.0060668

                                                                              
1272892109⋅x₁⋅x₂⋅x₄ + 0.878913909506331⋅x₁⋅x₂⋅x₅ - 0.00117290874335608⋅x₁⋅x₂ +

                          2                                                   
 2.37130667428623e-8⋅x₁⋅x₃  + 3.54390102928186e-6⋅x₁⋅x₃⋅x₄ + 9.25198602766616e

                                                                   2          
-6⋅x₁⋅x₃⋅x₅ + 1.55426160933231e-5⋅x₁⋅x₃ - 3.45800755208511e-6⋅x₁⋅x₄  - 8.25572

                   

In [34]:
import sympy as sp
x1, x2, x3, x4, x5 = sp.symbols('x1 x2 x3 x4 x5')
filtered_expression8 = filtered_ex
expr1 = filtered_expression8
exprpx5 =  x5
polynomialpx5 = sp.Poly(exprpx5, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px5 = polynomialpx5.monoms()
coeffs_px5 = polynomialpx5.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px5 = {sp.Mul(*[s**e for s, e in zip([x5], term)]): coeff
                          for term, coeff in zip(terms_px5, coeffs_px5)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px5.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx3 =  x3
polynomialpx3 = sp.Poly(exprpx3, x3)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px3 = polynomialpx3.monoms()
coeffs_px3 = polynomialpx3.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3 = {sp.Mul(*[s**e for s, e in zip([x3], term)]): coeff
                          for term, coeff in zip(terms_px3, coeffs_px3)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx4 =  x4
polynomialpx4 = sp.Poly(exprpx4, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px4 = polynomialpx5.monoms()
coeffs_px4 = polynomialpx4.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px4 = {sp.Mul(*[s**e for s, e in zip([x4], term)]): coeff
                          for term, coeff in zip(terms_px4, coeffs_px4)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px4.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx52 =  x5**2
polynomialpx52 = sp.Poly(exprpx52, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px52 = polynomialpx52.monoms()
coeffs_px52 = polynomialpx52.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px52 = {sp.Mul(*[s**e for s, e in zip([x5], term)]): coeff
                          for term, coeff in zip(terms_px52, coeffs_px52)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px52.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx53 =  x5**3
polynomialpx53 = sp.Poly(exprpx53, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px53 = polynomialpx53.monoms()
coeffs_px53 = polynomialpx53.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px53 = {sp.Mul(*[s**e for s, e in zip([x5], term)]): coeff
                          for term, coeff in zip(terms_px53, coeffs_px53)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px53.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx3x42 =  (x3 + x4)**2
polynomialpx3x42 = sp.Poly(exprpx3x42, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px3x42 = polynomialpx3x42.monoms()
coeffs_px3x42 = polynomialpx3x42.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x42 = {sp.Mul(*[s**e for s, e in zip([x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x42, coeffs_px3x42)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x42.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx3x43 =  (x3 + x4)**3
polynomialpx3x43 = sp.Poly(exprpx3x43, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px3x43 = polynomialpx3x43.monoms()
coeffs_px3x43 = polynomialpx3x43.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x43 = {sp.Mul(*[s**e for s, e in zip([x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x43, coeffs_px3x43)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x43.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx3x4x52 =  x5*(x3 + x4)**2
polynomialpx3x4x52 = sp.Poly(exprpx3x4x52, x5, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px3x4x52 = polynomialpx3x4x52.monoms()
coeffs_px3x4x52 = polynomialpx3x4x52.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x4x52 = {sp.Mul(*[s**e for s, e in zip([x5, x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x4x52, coeffs_px3x4x52)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x4x52.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx3x4x5 =  x5*(x3 + x4)
polynomialpx3x4x5 = sp.Poly(exprpx3x4x5, x5, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px3x4x5 = polynomialpx3x4x5.monoms()
coeffs_px3x4x5 = polynomialpx3x4x5.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x4x5 = {sp.Mul(*[s**e for s, e in zip([x5, x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x4x5, coeffs_px3x4x5)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x4x5.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
exprpx3x4x522 =  x5**2*(x3 + x4)
polynomialpx3x4x522 = sp.Poly(exprpx3x4x522, x5, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px3x4x522 = polynomialpx3x4x522.monoms()
coeffs_px3x4x522 = polynomialpx3x4x522.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x4x522 = {sp.Mul(*[s**e for s, e in zip([x5, x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x4x522, coeffs_px3x4x522)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x4x522.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
polynomialpx5 = sp.Poly(exprpx5, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px5 = polynomialpx5.monoms()
coeffs_px5 = polynomialpx5.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px5 = {sp.Mul(*[s**e for s, e in zip([x5], term)]): coeff
                          for term, coeff in zip(terms_px5, coeffs_px5)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px5.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx5 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx5
polynomialpx3 = sp.Poly(exprpx3, x3)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px3 = polynomialpx3.monoms()
coeffs_px3 = polynomialpx3.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3 = {sp.Mul(*[s**e for s, e in zip([x3], term)]): coeff
                          for term, coeff in zip(terms_px3, coeffs_px3)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx3 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx3
polynomialpx4 = sp.Poly(exprpx4, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px4 = polynomialpx4.monoms()
coeffs_px4 = polynomialpx4.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px4 = {sp.Mul(*[s**e for s, e in zip([x4], term)]): coeff
                          for term, coeff in zip(terms_px4, coeffs_px4)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px4.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx4 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx4
polynomialpx52 = sp.Poly(exprpx52, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px52 = polynomialpx52.monoms()
coeffs_px52 = polynomialpx52.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px52 = {sp.Mul(*[s**e for s, e in zip([x5], term)]): coeff
                          for term, coeff in zip(terms_px52, coeffs_px52)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px52.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx52 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx52
polynomialpx53 = sp.Poly(exprpx53, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px53 = polynomialpx53.monoms()
coeffs_px53 = polynomialpx53.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px53 = {sp.Mul(*[s**e for s, e in zip([x5], term)]): coeff
                          for term, coeff in zip(terms_px53, coeffs_px53)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px53.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx53 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx53
polynomialpx3x42 = sp.Poly(exprpx3x42, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px3x42 = polynomialpx3x42.monoms()
coeffs_px3x42 = polynomialpx3x42.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x42 = {sp.Mul(*[s**e for s, e in zip([x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x42, coeffs_px3x42)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x42.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx3x42 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx3x42
polynomialpx3x43 = sp.Poly(exprpx3x43, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px3x43 = polynomialpx3x43.monoms()
coeffs_px3x43 = polynomialpx3x43.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x43 = {sp.Mul(*[s**e for s, e in zip([x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x43, coeffs_px3x43)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x43.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx3x43 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx3x43
polynomialpx3x4x52 = sp.Poly(exprpx3x4x52, x5, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px3x4x52 = polynomialpx3x4x52.monoms()
coeffs_px3x4x52 = polynomialpx3x4x52.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x4x52 = {sp.Mul(*[s**e for s, e in zip([x5, x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x4x52, coeffs_px3x4x52)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x4x52.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx3x4x52 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx3x4x52
polynomialpx3x4x5 = sp.Poly(exprpx3x4x5, x5, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px3x4x5 = polynomialpx3x4x5.monoms()
coeffs_px3x4x5 = polynomialpx3x4x5.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x4x5 = {sp.Mul(*[s**e for s, e in zip([x5, x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x4x5, coeffs_px3x4x5)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x4x5.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx3x4x5 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx3x4x5
polynomialpx3x4x522 = sp.Poly(exprpx3x4x522, x5, x3, x4)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)

terms_px3x4x522 = polynomialpx3x4x522.monoms()
coeffs_px3x4x522 = polynomialpx3x4x522.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px3x4x522 = {sp.Mul(*[s**e for s, e in zip([x5, x3, x4], term)]): coeff
                          for term, coeff in zip(terms_px3x4x522, coeffs_px3x4x522)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px3x4x522.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx3x4x522 = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx3x4x522
expr1 = filtered_expression8
exprpx5f = filtered_expression8
polynomialpx5f = sp.Poly(exprpx5f, x1, x2, x3, x4, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px5f = polynomialpx5f.monoms()
coeffs_px5f = polynomialpx5f.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px5f = {sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)]): coeff
                          for term, coeff in zip(terms_px5f, coeffs_px5f)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px5f.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
for term, coeff in terms_with_coeffs1.items():
    print(f"{term}: {coeff}")
vp1 = list(terms_with_coeffs1.keys())
print("vp1 =", vp1)
polynomialpx5f = sp.Poly(exprpx5f, x1, x2, x3, x4, x5)
polynomial1 = sp.Poly(expr1, x1, x2, x3, x4, x5)
terms_px5f = polynomialpx5f.monoms()
coeffs_px5f = polynomialpx5f.coeffs()
terms1 = polynomial1.monoms()
coeffs1 = polynomial1.coeffs()
terms_with_coeffs_px5f = {sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)]): coeff
                          for term, coeff in zip(terms_px5f, coeffs_px5f)}
terms_with_coeffs1 = {}
for term in terms1:
    term_expr = sp.Mul(*[s**e for s, e in zip([x1, x2, x3, x4, x5], term)])
    coeff = terms_with_coeffs_px5f.get(term_expr, 0)
    terms_with_coeffs1[term_expr] = coeff
coeffs_column_matrixx5f = sp.Matrix([coeff for coeff in terms_with_coeffs1.values()])
coeffs_column_matrixx5f
combined_matrixp6 = sp.Matrix.hstack(coeffs_column_matrixx5, coeffs_column_matrixx3, coeffs_column_matrixx4, coeffs_column_matrixx52, coeffs_column_matrixx53, coeffs_column_matrixx3x42, coeffs_column_matrixx3x43, coeffs_column_matrixx3x4x52, coeffs_column_matrixx3x4x5, coeffs_column_matrixx3x4x522, coeffs_column_matrixx5f)
combined_matrixp6


x1**2*x3: 0
x1**2*x4: 0
x1**2*x5: 0
x1*x2*x3: 0
x1*x2*x4: 0
x1*x2*x5: 0
x1*x2: 0
x2**2*x3: 0
x2**2*x4: 0
x2**2*x5: 0
x3**3: 0
x3**2*x4: 0
x3**2*x5: 0
x3**2: 0
x3*x4**2: 0
x3*x4*x5: 0
x3*x4: 0
x3*x5**2: 0
x3*x5: 0
x3: 0
x4**3: 0
x4**2*x5: 0
x4**2: 0
x4*x5**2: 0
x4*x5: 0
x4: 0
x5**3: 0
x5**2: 0
x5: 1
1: 0
x1**2*x3: 0
x1**2*x4: 0
x1**2*x5: 0
x1*x2*x3: 0
x1*x2*x4: 0
x1*x2*x5: 0
x1*x2: 0
x2**2*x3: 0
x2**2*x4: 0
x2**2*x5: 0
x3**3: 0
x3**2*x4: 0
x3**2*x5: 0
x3**2: 0
x3*x4**2: 0
x3*x4*x5: 0
x3*x4: 0
x3*x5**2: 0
x3*x5: 0
x3: 1
x4**3: 0
x4**2*x5: 0
x4**2: 0
x4*x5**2: 0
x4*x5: 0
x4: 0
x5**3: 0
x5**2: 0
x5: 0
1: 0
x1**2*x3: 0
x1**2*x4: 0
x1**2*x5: 0
x1*x2*x3: 0
x1*x2*x4: 0
x1*x2*x5: 0
x1*x2: 0
x2**2*x3: 0
x2**2*x4: 0
x2**2*x5: 0
x3**3: 0
x3**2*x4: 0
x3**2*x5: 0
x3**2: 0
x3*x4**2: 0
x3*x4*x5: 0
x3*x4: 0
x3*x5**2: 0
x3*x5: 0
x3: 0
x4**3: 0
x4**2*x5: 0
x4**2: 0
x4*x5**2: 0
x4*x5: 0
x4: 1
x5**3: 0
x5**2: 0
x5: 0
1: 0
x1**2*x3: 0
x1**2*x4: 0
x1**2*x5: 0
x1*x2*x3: 0
x1*x2*x4: 0
x1*x2*x5: 0
x1*x2: 0
x2**

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0.0030216529125727],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0.00303707801716749],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   -0.439457323297892],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.00605189667404435],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.00606681272892109],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,    0.878913909506331],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.00117290874335608],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0.00303003438542142],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0.00302921563523044],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   -0.439457237738193],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0.00807453275443615],
[0, 0, 0, 0, 0, 0, 3, 0, 0, 0,   0.0223430803492342],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0,   -0.391045162526636],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0,   -0.174938873805118],
[0, 0, 0, 0, 0, 0, 3, 0, 0, 0,   0.0223580055674962],
[0, 0, 0, 0, 0, 0, 0, 2, 0, 0,   -0.780338240436667],
[0, 0, 0, 0, 0, 2, 0, 0, 0, 0,   0.0866643573641039],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1,   -0.529618572677951],
[0, 0, 0, 0, 0, 0, 

In [35]:
import numpy as np
import cvxpy as cp

C = combined_matrixp6
a = cp.Variable(11)
Ca = C @ a
objective = cp.Minimize(cp.norm1(Ca))
constraints = [
    cp.sum(a) == 1
]
problem = cp.Problem(objective, constraints)
problem.solve()
a_optimal = a.value
print(f"The optimal value of a is: [{a_optimal[0]}, {a_optimal[1]}, {a_optimal[2]}, {a_optimal[3]}, {a_optimal[4]}, {a_optimal[5]}, {a_optimal[6]}, {a_optimal[7]}, {a_optimal[8]}, {a_optimal[9]}, {a_optimal[10]}[")
print("The minimum value of ||C * a||_1 is:", problem.value)

The optimal value of a is: [0.2000002702936807, 0.19999979312487623, 0.19999979328244416, 0.20000035510281453, 0.19999978816476258, 2.1960992967273087e-12, 4.835386073544243e-13, 5.1412843156767594e-12, 4.5286713488545115e-12, 9.998976562379278e-12, 9.073157002440932e-12[
The minimum value of ||C * a||_1 is: 1.000000000033445


In [36]:
import numpy as np
C = combined_matrixp6
a = np.array(a_optimal)
resultp = np.dot(C, a)
formatted_resultp = ', '.join(map(str, resultp))
print(formatted_resultp)

2.74159312826550e-14, 2.75558856784226e-14, -3.98726529015422e-12, -5.49098086861545e-14, -5.50451443939082e-14, 7.97452389258010e-12, -1.06419851780054e-14, 2.74919777017232e-14, 2.74845490526946e-14, -3.98726451385763e-12, 5.56800110756775e-13, 1.65333809799003e-12, 1.59327016102756e-12, 6.08851428863272e-13, 1.65347351683861e-12, 3.20243726086314e-12, 5.17851791433478e-12, 5.19366410106356e-12, 5.19368605007722e-12, 0.199999793132148, 5.56751330893138e-13, 1.59325750286155e-12, 6.08859432797118e-13, 5.19367231872156e-12, 5.19365031617333e-12, 0.199999793289716, 0.199999788172143, 0.200000355094230, 0.200000270287495, 3.55236289301759e-12


In [37]:
import ast
v3_str = '[' + ', '.join(f'{x:.15g}' for x in resultp) + ']'
print("v3 (string):", v3_str)
v3_list = ast.literal_eval(v3_str)
v3_sympy = [sp.Float(x) for x in v3_list]
v1 = vp1
dot_product1 = sum(sp.Mul(v1_i, v3_i) for v1_i, v3_i in zip(v1, v3_sympy))
dot_product1_simplified = sp.simplify(dot_product1)
print("Dot Product1 (symbolic):", dot_product1_simplified)

v3 (string): [2.74159312826550e-14, 2.75558856784226e-14, -3.98726529015422e-12, -5.49098086861545e-14, -5.50451443939082e-14, 7.97452389258010e-12, -1.06419851780054e-14, 2.74919777017232e-14, 2.74845490526946e-14, -3.98726451385763e-12, 5.56800110756775e-13, 1.65333809799003e-12, 1.59327016102756e-12, 6.08851428863272e-13, 1.65347351683861e-12, 3.20243726086314e-12, 5.17851791433478e-12, 5.19366410106356e-12, 5.19368605007722e-12, 0.199999793132148, 5.56751330893138e-13, 1.59325750286155e-12, 6.08859432797118e-13, 5.19367231872156e-12, 5.19365031617333e-12, 0.199999793289716, 0.199999788172143, 0.200000355094230, 0.200000270287495, 3.55236289301759e-12]
Dot Product1 (symbolic): 2.7415931282655e-14*x1**2*x3 + 2.75558856784226e-14*x1**2*x4 - 3.98726529015422e-12*x1**2*x5 - 5.49098086861545e-14*x1*x2*x3 - 5.50451443939082e-14*x1*x2*x4 + 7.9745238925801e-12*x1*x2*x5 - 1.06419851780054e-14*x1*x2 + 2.74919777017232e-14*x2**2*x3 + 2.74845490526946e-14*x2**2*x4 - 3.98726451385763e-12*x2**2*x

In [38]:
# use authomatically like this instead the following code
import numpy as np
import cvxpy as cp
import sympy as sp
combined_matrixp6 = combined_matrixp6 = sp.Matrix.hstack(coeffs_column_matrixx5, coeffs_column_matrixx3, coeffs_column_matrixx4, coeffs_column_matrixx52, coeffs_column_matrixx53, coeffs_column_matrixx3x42, coeffs_column_matrixx3x43, coeffs_column_matrixx3x4x52, coeffs_column_matrixx3x4x5, coeffs_column_matrixx3x4x522, coeffs_column_matrixx5f)
C = np.array(combined_matrixp6).astype(np.float64)
a = cp.Variable(11)
Ca = C @ a
objective = cp.Minimize(cp.norm1(Ca))
constraints = [cp.sum(a) == 1]
problem = cp.Problem(objective, constraints)
problem.solve()
a_optimal = a.value
print(f"The optimal value of a is: [{a_optimal[0]}, {a_optimal[1]}, {a_optimal[2]}, {a_optimal[3]}, {a_optimal[4]}, {a_optimal[5]}, {a_optimal[6]}, {a_optimal[7]}, {a_optimal[8]}, {a_optimal[9]}, {a_optimal[10]}[")
resultp = np.dot(C, a_optimal)
final_result = np.dot(resultp, C)
print(final_result)

The optimal value of a is: [0.2000002702936807, 0.19999979312487623, 0.19999979328244416, 0.20000035510281453, 0.19999978816476258, 2.1960992967273087e-12, 4.835386073544243e-13, 5.1412843156767594e-12, 4.5286713488545115e-12, 9.998976562379278e-12, 9.073157002440932e-12[
[2.00000270e-01 1.99999793e-01 1.99999793e-01 2.00000355e-01
 1.99999788e-01 1.15747467e-11 1.10339863e-11 9.59140219e-12
 1.03873364e-11 1.03873364e-11 1.57713961e-01]


In [39]:
import numpy as np
import sympy as sp
combined_matrixp6 = combined_matrixp6 = sp.Matrix.hstack(coeffs_column_matrixx5, coeffs_column_matrixx3, coeffs_column_matrixx4, coeffs_column_matrixx52, coeffs_column_matrixx53, coeffs_column_matrixx3x42, coeffs_column_matrixx3x43, coeffs_column_matrixx3x4x52, coeffs_column_matrixx3x4x5, coeffs_column_matrixx3x4x522, coeffs_column_matrixx5f)
C = np.array(combined_matrixp6).astype(np.float64)
a = np.array(a_optimal)
resultp = np.dot(C, a)
final_result = np.dot(resultp, C)
print(final_result)

[2.00000270e-01 1.99999793e-01 1.99999793e-01 2.00000355e-01
 1.99999788e-01 1.15747467e-11 1.10339863e-11 9.59140219e-12
 1.03873364e-11 1.03873364e-11 1.57713961e-01]


In [40]:
# use authomatically like this instead the following code
import numpy as np
import cvxpy as cp
import sympy as sp
combined_matrixp6 = combined_matrixp6 = sp.Matrix.hstack(coeffs_column_matrixx5, coeffs_column_matrixx3, coeffs_column_matrixx4, coeffs_column_matrixx52, coeffs_column_matrixx53, coeffs_column_matrixx3x42, coeffs_column_matrixx3x43, coeffs_column_matrixx3x4x52, coeffs_column_matrixx3x4x5, coeffs_column_matrixx3x4x522, coeffs_column_matrixx5f)
C = np.array(combined_matrixp6).astype(np.float64)
a1 = cp.Variable(11)
Ca1 = C @ a1
objective1 = cp.Minimize(cp.norm1(Ca1))
constraints1 = [cp.sum(a1) == 1]
problem1 = cp.Problem(objective1, constraints1)
problem1.solve()
a_optimal1 = a1.value
resultp = np.dot(C, a_optimal1)
final_result = np.dot(resultp, C)
a2 = cp.Variable(11)
Ca2 = C @ a2
objective2 = cp.Minimize(cp.norm1(Ca2))
constraints2 = [
    cp.sum(a2) == 1,
    final_result[0] * a2[0] + final_result[1] * a2[1] + final_result[2] * a2[2] + final_result[3] * a2[3] + final_result[4] * a2[4] + final_result[5] * a2[5] + final_result[6] * a2[6] + final_result[7] * a2[7] + final_result[8] * a2[8] + final_result[9] * a2[9] + final_result[10] * a2[10] == 0
]
problem2 = cp.Problem(objective2, constraints2)
problem2.solve()
a_optimal2 = a2.value
print("The optimal value of a is:", a_optimal2)
print("The minimum value of ||C * a||_1 is:", problem2.value)

The optimal value of a is: [ 2.38476124e-10 -2.95633329e-10 -2.95632768e-10  3.33641090e-10
 -2.99947949e-10  7.76255803e-11  1.51398136e-11  2.05663857e-10
  4.99999582e-01  5.00000418e-01  3.60081254e-10]
The minimum value of ||C * a||_1 is: 2.0000000004341385


In [41]:
# use authomatically like this instead the following code
import numpy as np
import cvxpy as cp
import sympy as sp
combined_matrixp6 = combined_matrixp6 = sp.Matrix.hstack(coeffs_column_matrixx5, coeffs_column_matrixx3, coeffs_column_matrixx4, coeffs_column_matrixx52, coeffs_column_matrixx53, coeffs_column_matrixx3x42, coeffs_column_matrixx3x43, coeffs_column_matrixx3x4x52, coeffs_column_matrixx3x4x5, coeffs_column_matrixx3x4x522, coeffs_column_matrixx5f)
C = np.array(combined_matrixp6).astype(np.float64)
a1 = cp.Variable(11)
Ca1 = C @ a1
objective1 = cp.Minimize(cp.norm1(Ca1))
constraints1 = [cp.sum(a1) == 1]
problem1 = cp.Problem(objective1, constraints1)
problem1.solve()
a_optimal1 = a1.value
resultp = np.dot(C, a_optimal1)
final_result = np.dot(resultp, C)
a2 = cp.Variable(11)
Ca2 = C @ a2
objective2 = cp.Minimize(cp.norm1(Ca2))
constraints2 = [
    cp.sum(a2) == 1,
    final_result[0] * a2[0] + final_result[1] * a2[1] + final_result[2] * a2[2] + final_result[3] * a2[3] + final_result[4] * a2[4] + final_result[5] * a2[5] + final_result[6] * a2[6] + final_result[7] * a2[7] + final_result[8] * a2[8] + final_result[9] * a2[9] + final_result[10] * a2[10] == 0
]
problem2 = cp.Problem(objective2, constraints2)
problem2.solve()
a_optimal2 = a2.value
resultp_final = np.dot(C, a_optimal2)
formatted_resultp = ', '.join(map(str, resultp_final))
print("Formatted resultp:", formatted_resultp)

Formatted resultp: 1.0880405691946035e-12, 1.0935948601965148e-12, -1.582403439482632e-10, -2.1791745420316263e-12, -2.1845455337728343e-12, 3.1648042248443204e-10, -4.223424508567783e-13, 1.091058580446348e-12, 1.0907637638510614e-12, -1.582403131398195e-10, 1.8047301444966467e-11, 5.34647650866765e-11, 6.485582456643688e-11, 1.4633371245292184e-11, 5.347013937798096e-11, 1.3034254195831906e-10, 1.864573709699112e-10, 0.5000004174800511, 0.49999958201849726, -7.028920098182123e-12, 1.8045365546088903e-11, 6.485532220895706e-11, 1.463368889289086e-11, 0.5000004174800514, 0.49999958201849587, -7.0289202800626475e-12, -7.0289144113746065e-12, -7.029563161455122e-12, -7.029466117365519e-12, 1.4098061832209032e-10


In [42]:
import numpy as np
C = combined_matrixp6
a = np.array(a_optimal2)
resultp = np.dot(C, a)
formatted_resultp = ', '.join(map(str, resultp))
print(formatted_resultp)

1.08804056919460e-12, 1.09359486019651e-12, -1.58240343948263e-10, -2.17917454203163e-12, -2.18454553377283e-12, 3.16480422484432e-10, -4.22342450856778e-13, 1.09105858044635e-12, 1.09076376385106e-12, -1.58240313139820e-10, 1.80473014449665e-11, 5.34647650866765e-11, 6.48558245664369e-11, 1.46333712452922e-11, 5.34701393779810e-11, 1.30342541958319e-10, 1.86457370969911e-10, 0.500000417480051, 0.499999582018497, -7.02892009818212e-12, 1.80453655460889e-11, 6.48553222089571e-11, 1.46336888928909e-11, 0.500000417480051, 0.499999582018496, -7.02892028006265e-12, -7.02891441137461e-12, -7.02956316145512e-12, -7.02946611736552e-12, 1.40980618322090e-10


In [43]:
import ast
v3_str = '[' + ', '.join(f'{x:.15g}' for x in resultp) + ']'
print("v3 (string):", v3_str)
v3_list = ast.literal_eval(v3_str)
v3_sympy = [sp.Float(x) for x in v3_list]
v1 = vp1
dot_product1 = sum(sp.Mul(v1_i, v3_i) for v1_i, v3_i in zip(v1, v3_sympy))
dot_product1_simplified = sp.simplify(dot_product1)
print("Dot Product1 (symbolic):", dot_product1_simplified)

v3 (string): [1.08804056919460e-12, 1.09359486019651e-12, -1.58240343948263e-10, -2.17917454203163e-12, -2.18454553377283e-12, 3.16480422484432e-10, -4.22342450856778e-13, 1.09105858044635e-12, 1.09076376385106e-12, -1.58240313139820e-10, 1.80473014449665e-11, 5.34647650866765e-11, 6.48558245664369e-11, 1.46333712452922e-11, 5.34701393779810e-11, 1.30342541958319e-10, 1.86457370969911e-10, 0.500000417480051, 0.499999582018497, -7.02892009818212e-12, 1.80453655460889e-11, 6.48553222089571e-11, 1.46336888928909e-11, 0.500000417480051, 0.499999582018496, -7.02892028006265e-12, -7.02891441137461e-12, -7.02956316145512e-12, -7.02946611736552e-12, 1.40980618322090e-10]
Dot Product1 (symbolic): 1.0880405691946e-12*x1**2*x3 + 1.09359486019651e-12*x1**2*x4 - 1.58240343948263e-10*x1**2*x5 - 2.17917454203163e-12*x1*x2*x3 - 2.18454553377283e-12*x1*x2*x4 + 3.16480422484432e-10*x1*x2*x5 - 4.22342450856778e-13*x1*x2 + 1.09105858044635e-12*x2**2*x3 + 1.09076376385106e-12*x2**2*x4 - 1.5824031313982e-10